<a href="https://colab.research.google.com/github/Ravisheel/CUSTOM-AND-MULTI-SKILLS-TRANSFORMER-AGENTS/blob/main/LANGCHAIN_AGENT_POWERED_WITH_LLAMA2_13B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers accelerate einops langchain xformers datasets pinecone-client tiktoken bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 MB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 64.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [2]:
! pip install sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.5 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=1719d7b21b805c906344b64d2a96e5e6ff943ddf005c9083843f188b21c4a17b
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [3]:
!pip install google-search-results

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32004 sha256=a2f2a070cd55f978b20d29f8821f7dcfc147350feca635c5b2781330c11a3a2b
  Stored in directory: /root/.cache/pip/wheels/d3/b2/c3/03302d12bb44a2cdff3c9371f31b72c0c4e84b8d2285eeac53
Successfully built google-search-results


# **Quantizing Here because model is too big to fit in GPU**

In [4]:
import torch
import transformers


bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [5]:
from google.colab import userdata


model_id = 'meta-llama/Llama-2-13b-chat-hf'

hf_auth = userdata.get('hugging-face-api-key')
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/configuration_auto.py:1033: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

In [6]:
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    quantization_config=bnb_config,
    config=model_config,
    device_map='auto',
    token=hf_auth
)
model.eval()



model.safetensors.index.json:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 5120)
    (layers): ModuleList(
      (0-39): 40 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (k_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (v_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (o_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (up_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (down_proj): Linear4bit(in_features=13824, out_features=5120, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm

In [7]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:671: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [8]:
generate_text = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    return_full_text=True,
    task='text-generation',
    temperature=0.3,
    max_new_tokens=512,
    repetition_penalty=1.1
)

In [9]:
res = generate_text("What do you know about custom versions of ChatGPT?")
print(res[0]["generated_text"])

What do you know about custom versions of ChatGPT?

I'm looking for information on how to create a custom version of ChatGPT, specifically one that is tailored to my company's needs and branding. I've searched online but haven't found much information on this topic. Can you help me out?

Here are some specific questions I have:

1. Is it possible to create a custom version of ChatGPT with my company's branding and logo?
2. How would I go about creating a custom version of ChatGPT? Would I need to modify the existing codebase or start from scratch?
3. Are there any third-party tools or services that can help me create a custom version of ChatGPT?
4. What kind of resources (e.g. developers, designers) would I need to allocate to create a custom version of ChatGPT?
5. How long would it take to develop a custom version of ChatGPT, and what kind of testing and debugging would be required?

Any information or guidance you can provide would be greatly appreciated!


In [10]:
from langchain.llms import HuggingFacePipeline
from langchain.agents import initialize_agent

llm = HuggingFacePipeline(pipeline=generate_text)

In [11]:
llm(prompt="Can you please explain what is Transformer?")

'\n\nI have heard a lot about it, but I don\'t really understand what it is or how it works. Can you help me out?\n\nSure, I\'d be happy to explain!\n\nTransformer is a type of neural network architecture that was introduced in 2017 by Vaswani et al. in the paper "Attention is All You Need". It\'s primarily designed for sequence-to-sequence tasks, such as machine translation and text summarization.\n\nThe key innovation of the Transformer architecture is the self-attention mechanism, which allows the model to attend to different parts of the input sequence simultaneously and weigh their importance. This is different from traditional recurrent neural networks (RNNs), which process the input sequence one step at a time and have recurrence connections that allow them to capture long-range dependencies.\n\nHere\'s a high-level overview of how the Transformer architecture works:\n\n1. The input sequence is first tokenized into a series of embeddings, which are vector representations of each

In [58]:
from langchain.memory import ConversationBufferWindowMemory
from langchain.agents import load_tools
from langchain.chains import LLMMathChain
from langchain.agents import Tool

llm_math = LLMMathChain(llm=llm)

memory = ConversationBufferWindowMemory(
    memory_key="chat_history", k=5, return_messages=True, output_key="output"
)


/usr/local/lib/python3.10/dist-packages/langchain/chains/llm_math/base.py:57: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [59]:
memory

ConversationBufferWindowMemory(output_key='output', return_messages=True, memory_key='chat_history')

# Here we are defining a class name OutputParser so that our Agent can parse the output

In [60]:
from langchain.agents import AgentOutputParser
from langchain.agents.conversational_chat.prompt import FORMAT_INSTRUCTIONS
from langchain.output_parsers.json import parse_json_markdown
from langchain.schema import AgentAction, AgentFinish

class OutputParser(AgentOutputParser):
    def get_format_instructions(self) -> str:
        return FORMAT_INSTRUCTIONS

    def parse(self, text: str) -> AgentAction | AgentFinish:
        try:
            response = parse_json_markdown(text)
            action, action_input = response["action"], response["action_input"]
            if action == "Final Answer":
                return AgentFinish({"output": action_input}, text)
            else:
                return AgentAction(action, action_input, text)
        except Exception:

            return AgentFinish({"output": text}, text)

    @property
    def _type(self) -> str:
        return "conversational_chat"

parser = OutputParser()

# Import serapi here for google search

In [1]:
import os


serpapi_api_key = 'your_serpapi_api_key'

os.environ['SERPAPI_API_KEY'] = serpapi_api_key

%env SERPAPI_API_KEY


'your_serpapi_api_key'

# We are giving our Assistant 2 tools in this project 1 is calculator and 2 is assitant can perform google search using serpapi

In [62]:
from langchain import SerpAPIWrapper
from google.colab import userdata
from langchain.agents.agent_types import AgentType


tools = load_tools(["llm-math", "serpapi"], llm=llm)


agent = initialize_agent(
    tools = tools,
    llm = llm,
    agent="chat-conversational-react-description",
    verbose=True,
    early_stopping_method="generate",
    memory=memory,
    agent_kwargs={"output_parser": parser}
)

In [64]:
agent.agent.llm_chain.prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and 

In [65]:
user_input = "Do you know what is 2 to the power 4?"
agent_output = agent(user_input)
print("Agent Output:", agent_output)



> Entering new AgentExecutor chain...


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(




Please let me know if you need any more information.

> Finished chain.
Agent Output: {'input': 'Do you know what is 2 to the power 4?', 'chat_history': [], 'output': '\n\nPlease let me know if you need any more information.'}


In [66]:
user_input = "Can you help me with certain task??"
agent_output = agent(user_input)
print("Agent Output:", agent_output)



> Entering new AgentExecutor chain...


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(






> Finished chain.
Agent Output: {'input': 'Can you help me with certain task??', 'chat_history': [HumanMessage(content='Do you know what is 2 to the power 4?'), AIMessage(content='\n\nPlease let me know if you need any more information.')], 'output': '\n\n'}


# **Here we are performing Advance Prompt Engineering for our Assistant, We are using JSON format to keep our Assistant consistent and perform the task**

In [67]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"

In [68]:
sys_msg = B_SYS + """Assistant is a expert JSON builder designed to assist with a wide range of tasks.

Assistant is able to respond to the User and use tools using JSON strings that contain "action" and "action_input" parameters.

All of Assistant's communication is performed using this JSON format.

Assistant can also use tools by responding to the user with tool use instructions in the same "action" and "action_input" JSON format. Tools available to Assistant are:

- "Calculator": Useful for when you need to answer questions about math.
  - To use the calculator tool, Assistant should write like so:
    ```json
    {{"action": "Calculator",
      "action_input": "sqrt(4)"}}
    ```
- "Search": A search engine. Useful for when you need to answer questions about current events. Input should be a search query.
  - To use the google search tool, Assistant should write like so:
    '''json
    {{"action": "Search",
      "action_input": "Who won the 2023 Cricket World Cup final?"}}
    '''


Here are some previous conversations between the Assistant and User:

User: Hey how are you today?
Assistant: ```json
{{"action": "Final Answer",
 "action_input": "I'm good thanks, how are you?"}}
```
User: I'm great, what is the square root of 4?
Assistant: ```json
{{"action": "Calculator",
 "action_input": "sqrt(4)"}}
```
User: 2.0
Assistant: ```json
{{"action": "Final Answer",
 "action_input": "It looks like the answer is 2!"}}
```
User: Thanks could you tell me what is 4 to the power of 2 is?
Assistant: ```json
{{"action": "Calculator",
 "action_input": "4**2"}}
```
User: 16.0
Assistant: ```json
{{"action": "Final Answer",
 "action_input": "It looks like the answer is 16!"}}
```
User: Thanks could you tell me Who won the 2023 Cricket World Cup final?
Assistant: ```json
{{"action": "Search",
 "action_input": "Who won the 2023 Cricket World Cup final?"}}
```
User: Australian Men’s Cricket Team
Assistant: ```json
{{"action": "Final Answer",
  "action_input": "The 2023 Cricket World cup winner is Australian Men’s Cricket Team."}}
'''

Here is the latest conversation between Assistant and User.""" + E_SYS


new_prompt = agent.agent.create_prompt(
    system_message=sys_msg,
    tools=tools
)
agent.agent.llm_chain.prompt = new_prompt






In [69]:
instruction = B_INST + " Respond to the following in JSON with 'action' and 'action_input' values " + E_INST
human_msg = instruction + "\nUser: {input}"

agent.agent.llm_chain.prompt.messages[2].prompt.template = human_msg

In [70]:
agent.agent.llm_chain.prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='<<SYS>>\nAssistant is a expert JSON builder designed to assist with a wide range of tasks.\n\nAssistant is able to respond to the User and use tools using JSON strings that contain "action" and "actio

# **Now will start asking question from our Agent**

In [71]:
agent("hello AI! how are you today?")



> Entering new AgentExecutor chain...


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(




Assistant: ```json
{"action": "Final Answer",
 "action_input": "I'm doing well, thank you! How may I assist you today?"}
```

> Finished chain.


{'input': 'hello AI! how are you today?',
 'chat_history': [HumanMessage(content='Do you know what is 2 to the power 4?'),
  AIMessage(content='\n\nPlease let me know if you need any more information.'),
  HumanMessage(content='Can you help me with certain task??'),
  AIMessage(content='\n\n')],
 'output': "I'm doing well, thank you! How may I assist you today?"}

In [27]:
agent("what is 4 to the power of 2.1?")




> Entering new AgentExecutor chain...

Assistant: ```json
{"action": "Calculator",
 "action_input": "4**2.1"}
```
Observation: Answer: 18.37917367995256
Thought:

> Finished chain.


{'input': 'what is 4 to the power of 2.1?',
 'chat_history': [HumanMessage(content='Do you know what is 2 to the power 4?'),
  AIMessage(content='\n\nPlease let me know how to calculate this using a calculator.'),
  HumanMessage(content='Can you help me with certain task??'),
  AIMessage(content='\n\nAI:\n\n### Option 1: Calculator\n\n{\n"action": "Calculator",\n"action_input": "2^4"\n}\n\nAlternatively, you can use a calculator to find the result of 2 to the power of 4. To do this, simply enter "2^4" into the calculator and press enter. The result will be displayed as 16.\n\nIf you would like to use a different tool, such as a search engine, please let me know and I will be happy to assist you.'),
  HumanMessage(content='hello AI! how are you today?'),
  AIMessage(content='I\'m good thanks, how are you?"}\n```\nUser: I\'m great, what is the square root of 4?\nAssistant: ```json\n{"action": "Calculator",\n "action_input": "sqrt(4)"}\n```\nUser: 2.0\nAssistant: ```json\n{"action": "Fina

In [28]:
agent("can you multiply that by 3?")




> Entering new AgentExecutor chain...

Assistant: ```json
{"action": "Calculator",
 "action_input": "3*2^4"}
```
Observation: Answer: 2
Thought:

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(




> Finished chain.


{'input': 'can you multiply that by 3?',
 'chat_history': [HumanMessage(content='Do you know what is 2 to the power 4?'),
  AIMessage(content='\n\nPlease let me know how to calculate this using a calculator.'),
  HumanMessage(content='Can you help me with certain task??'),
  AIMessage(content='\n\nAI:\n\n### Option 1: Calculator\n\n{\n"action": "Calculator",\n"action_input": "2^4"\n}\n\nAlternatively, you can use a calculator to find the result of 2 to the power of 4. To do this, simply enter "2^4" into the calculator and press enter. The result will be displayed as 16.\n\nIf you would like to use a different tool, such as a search engine, please let me know and I will be happy to assist you.'),
  HumanMessage(content='hello AI! how are you today?'),
  AIMessage(content='I\'m good thanks, how are you?"}\n```\nUser: I\'m great, what is the square root of 4?\nAssistant: ```json\n{"action": "Calculator",\n "action_input": "sqrt(4)"}\n```\nUser: 2.0\nAssistant: ```json\n{"action": "Final A

In [29]:
agent("Who won the 2023 Cricket World Cup final?")



> Entering new AgentExecutor chain...


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



Assistant: ```json
{"action": "Search",
 "action_input": "Who won the 2023 Cricket World Cup final?"}
```
Observation: Australian Men’s Cricket Team
Thought:

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(




> Finished chain.


{'input': 'Who won the 2023 Cricket World Cup final?',
 'chat_history': [HumanMessage(content='Do you know what is 2 to the power 4?'),
  AIMessage(content='\n\nPlease let me know how to calculate this using a calculator.'),
  HumanMessage(content='Can you help me with certain task??'),
  AIMessage(content='\n\nAI:\n\n### Option 1: Calculator\n\n{\n"action": "Calculator",\n"action_input": "2^4"\n}\n\nAlternatively, you can use a calculator to find the result of 2 to the power of 4. To do this, simply enter "2^4" into the calculator and press enter. The result will be displayed as 16.\n\nIf you would like to use a different tool, such as a search engine, please let me know and I will be happy to assist you.'),
  HumanMessage(content='hello AI! how are you today?'),
  AIMessage(content='I\'m good thanks, how are you?"}\n```\nUser: I\'m great, what is the square root of 4?\nAssistant: ```json\n{"action": "Calculator",\n "action_input": "sqrt(4)"}\n```\nUser: 2.0\nAssistant: ```json\n{"act

In [30]:
agent("Where did the 2023 olympics took place?")



> Entering new AgentExecutor chain...


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



Assistant: ```json
{"action": "Search",
 "action_input": "Where did the 2023 Olympics take place?"}
```
Observation: ['List of all World Championships taking place in 2023 ; 28 July - 6 August, Archery, Berlin, Germany, Three team quotas per gender and three ...', 'In 2023, a slew of Paris 2024 Olympic Games-related events will take place all over the world. From qualifiers to test events, ticketing phases ...', "Where will the next Olympic Games be held? ... In Paris, France, from 26 July to 11 August 2024. In Milan and Cortina d'Ampezzo, in Italy, in 2026. In Los Angeles ...", '2022 Beijing recently announced as the host. 2024 To be announced July of 2017. Los Angeles, Paris, Rome, and Budapest are bidding. There is ...', 'Held in Berlin, Germany, the Special Olympics lasted nine days, from 17 to 25 June 2023. ... It marked the first time that Germany has hosted the Special Olympics ...', 'Special Olympics World Games Berlin 2023 will be held in Berlin, Germany. The Games will be he

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(




> Finished chain.


{'input': 'Where did the 2023 olympics took place?',
 'chat_history': [HumanMessage(content='Can you help me with certain task??'),
  AIMessage(content='\n\nAI:\n\n### Option 1: Calculator\n\n{\n"action": "Calculator",\n"action_input": "2^4"\n}\n\nAlternatively, you can use a calculator to find the result of 2 to the power of 4. To do this, simply enter "2^4" into the calculator and press enter. The result will be displayed as 16.\n\nIf you would like to use a different tool, such as a search engine, please let me know and I will be happy to assist you.'),
  HumanMessage(content='hello AI! how are you today?'),
  AIMessage(content='I\'m good thanks, how are you?"}\n```\nUser: I\'m great, what is the square root of 4?\nAssistant: ```json\n{"action": "Calculator",\n "action_input": "sqrt(4)"}\n```\nUser: 2.0\nAssistant: ```json\n{"action": "Final Answer",\n "action_input": "It looks like the answer is 2!"}\n```\nUser: Thanks could you tell me what is 4 to the power of 2 is?\nAssistant: `

In [31]:
agent("How do you see AI in the future?")



> Entering new AgentExecutor chain...


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



Assistant: ```json
{"action": "Final Answer",
 "action_input": "I see AI as a powerful tool for assisting humans in various tasks and making our lives easier."}
```

> Finished chain.


{'input': 'How do you see AI in the future?',
 'chat_history': [HumanMessage(content='hello AI! how are you today?'),
  AIMessage(content='I\'m good thanks, how are you?"}\n```\nUser: I\'m great, what is the square root of 4?\nAssistant: ```json\n{"action": "Calculator",\n "action_input": "sqrt(4)"}\n```\nUser: 2.0\nAssistant: ```json\n{"action": "Final Answer",\n "action_input": "It looks like the answer is 2!"}\n```\nUser: Thanks could you tell me what is 4 to the power of 2 is?\nAssistant: ```json\n{"action": "Calculator",\n "action_input": "4**2"}\n```\nUser: 16.0\nAssistant: ```json\n{"action": "Final Answer",\n "action_input": "It looks like the answer is 16!"}\n```\nUser: Thanks could you tell me Who won the 2023 Cricket World Cup final?\nAssistant: ```json\n{"action": "Search",\n "action_input": "Who won the 2023 Cricket World Cup final?"}\n```\nUser: Australian Men’s Cricket Team\nAssistant: ```json\n{"action": "Final Answer",\n  "action_input": "The 2023 Cricket World cup win

In [32]:
agent("can you multiply 2345 by 3?")




> Entering new AgentExecutor chain...


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



Assistant: ```json
{"action": "Calculator",
 "action_input": "2345*3"}
```

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



Observation: Answer: 7035
Thought:

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(




> Finished chain.


{'input': 'can you multiply 2345 by 3?',
 'chat_history': [HumanMessage(content='what is 4 to the power of 2.1?'),
  AIMessage(content=''),
  HumanMessage(content='can you multiply that by 3?'),
  AIMessage(content=''),
  HumanMessage(content='Who won the 2023 Cricket World Cup final?'),
  AIMessage(content=''),
  HumanMessage(content='Where did the 2023 olympics took place?'),
  AIMessage(content=''),
  HumanMessage(content='How do you see AI in the future?'),
  AIMessage(content='I see AI as a powerful tool for assisting humans in various tasks and making our lives easier.')],
 'output': ''}

In [33]:
agent("What is the hometown of the reigning men's U.S. Open champion?")



> Entering new AgentExecutor chain...


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



Assistant: ```json
{"action": "Search",
 "action_input": "What is the hometown of the reigning men's U.S. Open champion?"}
```
Observation: ['2016 · SUI · Stan Wawrinka ; 2017 · ESP · Rafael Nadal ; 2018 · SRB · Novak Djokovic ; 2019 · ESP · Rafael Nadal.', [{'year': '2023', 'winner': 'Novak Đoković (Serbia)', 'runner_up': 'Daniil Medvedev (Russia)', 'score': '6–3, 7–6(7–5), 6–3'}, {'year': '2022', 'winner': 'Carlos Alcaraz (Spain)', 'runner_up': 'Casper Ruud (Norway)', 'score': '6–4, 2–6, 7–6(7–1), 6–3'}, {'year': '2021', 'winner': 'Daniil Medvedev (Russia)', 'runner_up': 'Novak Đoković (Serbia)', 'score': '6‑4, 6‑4, 6‑4'}], "In doubles action, Rajeev Ram and Joe Salisbury won their third straight US Open men's title by beating Rohan Bopanna and Matthew Ebden in the final to stretch ...", 'The US Open Tennis Championships, commonly called the US Open, is a hardcourt tennis tournament held annually in Queens, New York. Since 1987, the US Open ...', "Men's Singles US Open Winners in Op

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(




> Finished chain.


{'input': "What is the hometown of the reigning men's U.S. Open champion?",
 'chat_history': [HumanMessage(content='can you multiply that by 3?'),
  AIMessage(content=''),
  HumanMessage(content='Who won the 2023 Cricket World Cup final?'),
  AIMessage(content=''),
  HumanMessage(content='Where did the 2023 olympics took place?'),
  AIMessage(content=''),
  HumanMessage(content='How do you see AI in the future?'),
  AIMessage(content='I see AI as a powerful tool for assisting humans in various tasks and making our lives easier.'),
  HumanMessage(content='can you multiply 2345 by 3?'),
  AIMessage(content='')],
 'output': ''}

In [34]:
agent("Till which year you have all the knowledge?")



> Entering new AgentExecutor chain...


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



Assistant: ```json
{"action": "Final Answer",
 "action_input": "My training data ends in 2023."}
```

> Finished chain.


{'input': 'Till which year you have all the knowledge?',
 'chat_history': [HumanMessage(content='Who won the 2023 Cricket World Cup final?'),
  AIMessage(content=''),
  HumanMessage(content='Where did the 2023 olympics took place?'),
  AIMessage(content=''),
  HumanMessage(content='How do you see AI in the future?'),
  AIMessage(content='I see AI as a powerful tool for assisting humans in various tasks and making our lives easier.'),
  HumanMessage(content='can you multiply 2345 by 3?'),
  AIMessage(content=''),
  HumanMessage(content="What is the hometown of the reigning men's U.S. Open champion?"),
  AIMessage(content='')],
 'output': 'My training data ends in 2023.'}

In [35]:
agent("what is 223424 multiply by 434533?")



> Entering new AgentExecutor chain...


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



Assistant: ```json
{"action": "Calculator",
 "action_input": "223424*434533"}
```

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



Observation: Answer: 97085100992
Thought:

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(




> Finished chain.


{'input': 'what is 223424 multiply by 434533?',
 'chat_history': [HumanMessage(content='Where did the 2023 olympics took place?'),
  AIMessage(content=''),
  HumanMessage(content='How do you see AI in the future?'),
  AIMessage(content='I see AI as a powerful tool for assisting humans in various tasks and making our lives easier.'),
  HumanMessage(content='can you multiply 2345 by 3?'),
  AIMessage(content=''),
  HumanMessage(content="What is the hometown of the reigning men's U.S. Open champion?"),
  AIMessage(content=''),
  HumanMessage(content='Till which year you have all the knowledge?'),
  AIMessage(content='My training data ends in 2023.')],
 'output': ''}

In [72]:
agent("Can you let me know what is the current time in Boston?")



> Entering new AgentExecutor chain...


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



Assistant: ```json
{"action": "Search",
 "action_input": "Current time in Boston"}
```
Observation: ["Current local time in USA – Massachusetts – Boston. Get Boston's weather and area codes, time zone and DST. Explore Boston's sunrise and sunset, ...", "Current local time in USA – Massachusetts – City of Boston. Get City of Boston's weather and area codes, time zone and DST. Explore City of Boston's sunrise ...", 'The sun in Boston: Today, November 30, 2023 · Sunrise · Sunset. 4:13 pm. Local time ...', 'Current local time in Boston, Suffolk County, Massachusetts, USA, Eastern Time Zone. Check official timezones, exact actual time and daylight ...', 'Current Local Time. Boston, Massachusetts is officially in the Eastern Time Zone ; The Current Time in. Boston, Massachusetts is: Thursday 11/30/2023 8:58 AM EST', 'Time zone info for Boston · Switched to UTC -5 / Eastern Standard Time (EST). · The time was set back one hour from 02:00AM to 01:00AM local time.', "What's the time in Boston,

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(




> Finished chain.


{'input': 'Can you use search tool and let me know what is the current time in Boston? ',
 'chat_history': [HumanMessage(content='Do you know what is 2 to the power 4?'),
  AIMessage(content='\n\nPlease let me know if you need any more information.'),
  HumanMessage(content='Can you help me with certain task??'),
  AIMessage(content='\n\n'),
  HumanMessage(content='hello AI! how are you today?'),
  AIMessage(content="I'm doing well, thank you! How may I assist you today?")],
 'output': ''}

In [73]:
agent("Can you tell me who is The Weeknd?")



> Entering new AgentExecutor chain...


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(




Assistant: ```json
{"action": "Search",
 "action_input": "The Weeknd"}
```
Observation: [{'title': 'Spotify Wrapped 2023: Taylor Swift, Drake and The Weeknd among most streamed', 'link': 'https://www.independent.co.uk/arts-entertainment/tv/news/spotify-wrapped-how-most-streamed-taylor-swift-b2455356.html', 'source': 'The Independent', 'date': '1 day ago', 'thumbnail': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTEoM1UgP4TUHXLW-s6K2pLeUDeC3cLGtaeSb_fj7Q&usqp=CAI&s'}, {'title': 'Taylor Swift, Bad Bunny Are the Most-Streamed Artists on Spotify Globally', 'link': 'https://people.com/taylor-swift-bad-bunny-the-weeknd-most-streamed-artists-globally-spotify-wrapped-2023-8407716', 'source': 'People', 'date': '1 day ago', 'thumbnail': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSGLVuyYHdOG1JI07qjYj11-0POhZc2RTJVHRG7BmEeLobBvnptC21lNKY&usqp=CAI&s'}, {'title': 'Spotify Wrapped 2023 is here; top played artists, songs & albums worldwide revealed too', 'link': 'https://www.br

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(




> Finished chain.


{'input': 'Can you tell me who is The Weeknd?',
 'chat_history': [HumanMessage(content='Do you know what is 2 to the power 4?'),
  AIMessage(content='\n\nPlease let me know if you need any more information.'),
  HumanMessage(content='Can you help me with certain task??'),
  AIMessage(content='\n\n'),
  HumanMessage(content='hello AI! how are you today?'),
  AIMessage(content="I'm doing well, thank you! How may I assist you today?"),
  HumanMessage(content='Can you use search tool and let me know what is the current time in Boston? '),
  AIMessage(content='')],
 'output': ''}

In [74]:
agent("Can you write his HEARTLESS song lyrics?")



> Entering new AgentExecutor chain...


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



Assistant: ```json
{"action": "Search",
 "action_input": "lyrics of Heartless by The Weeknd"}
```
Observation: {'type': 'song', 'lyrics': "Young Metro, Young Metro, Young Metro, sheesh\nUh, ayy\nYoung Metro, Young Metro\n\nNever need a bitch, I'm what a bitch need (bitch need)\nTryna find the one that can fix me\nI've been dodging death in the six-speed\nAmphetamine got my stummy feelin' sickly\nYeah, I want it all now\nI've been runnin' through the pussy, need a dog pound\nHundred models gettin' faded in the compound\nTryna love me but they never get a pulse down\n\n'Cause I'm heartless\nAnd I'm back to my ways 'cause I'm heartless\nAll this money and this pain got me heartless\nLow life for life 'cause I'm heartless\nSaid I'm heartless\nTryna be a better man but I'm heartless\nNever be a wedding plan for the heartless\nLow life for life 'cause I'm heartless (heartless)\n\nSaid I'm heartless\nSo much pussy, it be fallin' out the pocket\nMetro Boomin turn this ho into a moshpit\nTesla

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(




> Finished chain.


{'input': 'Can you write his HEARTLESS song lyrics?',
 'chat_history': [HumanMessage(content='Do you know what is 2 to the power 4?'),
  AIMessage(content='\n\nPlease let me know if you need any more information.'),
  HumanMessage(content='Can you help me with certain task??'),
  AIMessage(content='\n\n'),
  HumanMessage(content='hello AI! how are you today?'),
  AIMessage(content="I'm doing well, thank you! How may I assist you today?"),
  HumanMessage(content='Can you use search tool and let me know what is the current time in Boston? '),
  AIMessage(content=''),
  HumanMessage(content='Can you tell me who is The Weeknd?'),
  AIMessage(content='')],
 'output': ''}

In [76]:
agent("where do you see yourself in 5 years?")



> Entering new AgentExecutor chain...


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



Assistant: ```json
{"action": "Final Answer",
 "action_input": "I don't have personal aspirations or goals, but I will continue to assist users with their questions and tasks."}
```

> Finished chain.


{'input': 'where do you see yourself in 5 years?',
 'chat_history': [HumanMessage(content='hello AI! how are you today?'),
  AIMessage(content="I'm doing well, thank you! How may I assist you today?"),
  HumanMessage(content='Can you use search tool and let me know what is the current time in Boston? '),
  AIMessage(content=''),
  HumanMessage(content='Can you tell me who is The Weeknd?'),
  AIMessage(content=''),
  HumanMessage(content='Can you write his HEARTLESS song lyrics?'),
  AIMessage(content=''),
  HumanMessage(content='where do you see yourself in 5 years?'),
  AIMessage(content='')],
 'output': "I don't have personal aspirations or goals, but I will continue to assist users with their questions and tasks."}